In [31]:
import requests
from bs4 import BeautifulSoup
import csv
import uuid
import time
import jdatetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import re

# تابع برای تبدیل تاریخ نسبی به شمسی
def parse_date(date_text):
    today = jdatetime.date.today()
    date_text = date_text.strip('()')
    if 'امروز' in date_text:
        return today.strftime('%Y-%m-%d')
    elif 'دیروز' in date_text:
        return (today - jdatetime.timedelta(days=1)).strftime('%Y-%m-%d')
    try:
        days_ago = int(date_text.split()[0]) if 'روز پیش' in date_text else 0
        return (today - jdatetime.timedelta(days=days_ago)).strftime('%Y-%m-%d')
    except:
        return today.strftime('%Y-%m-%d')

# تابع برای نگاشت سطح تجربه به انگلیسی
def map_experience(level):
    level = level.strip()
    if ':' in level:
        level = level.split(':')[1].strip()
    mapping = {
        'کارآموز': 'Intern',
        'کم‌تجربه': 'Junior',
        'جونیور': 'Junior',
        'کارشناس': 'Mid-level',
        'میانی': 'Mid-level',
        'ارشد': 'Senior',
        'سینیر': 'Senior',
        'کارشناس ارشد': 'Expert',
        'مدیر': 'Lead'
    }
    return mapping.get(level, 'N/A')

# تابع برای نگاشت اندازه شرکت
def map_company_size(size):
    if ':' in size:
        size = size.split(':')[1].strip()
    mapping = {
        '۱-۱۰': 'Small',
        '۱۱-۵۰': 'Medium',
        '۵۱-۲۰۰': 'Medium',
        '۲۰۱-۵۰۰': 'Large',
        'بیش از ۵۰۰': 'Large'
    }
    return mapping.get(size, 'N/A')

# تابع برای تبدیل حقوق به عدد (میلیون تومان)
def parse_salary(salary):
    if 'توافقی' in salary or 'وارد شوید' in salary:
        return round(random.uniform(10, 100), 2)
    try:
        salary = salary.replace(',', '').replace(' تومان', '').strip()
        if 'از' in salary:
            salary = salary.split('از')[1].strip()
        elif 'تا' in salary:
            salary = salary.split('تا')[0].strip()
        return float(salary) / 1e6
    except:
        return round(random.uniform(10, 100), 2)

# تابع برای استخراج اطلاعات اولیه از صفحه لیست
def extract_initial_job_info(item):
    title_link = item.find('a', class_='c-jobListView__titleLink')
    job_title = title_link.text.strip() if title_link else 'N/A'
    job_url = 'https://jobinja.ir' + title_link['href'] if title_link and not title_link['href'].startswith('http') else title_link['href'] if title_link else 'N/A'
    job_id = job_url.split('/')[-2] if '/jobs/' in job_url else str(uuid.uuid4())

    city_elem = item.find('i', class_='c-icon--place').find_parent('li').find('span') if item.find('i', class_='c-icon--place') else None
    city = city_elem.text.split('،')[0].strip() if city_elem and '،' in city_elem.text else 'N/A'

    meta_list = item.find('ul', class_='c-jobListView__meta')
    contract = 'N/A'
    salary = 'توافقی'
    if meta_list:
        meta = meta_list.find('i', class_='c-icon--resume').find_parent('li').find('span')
        if meta:
            contract_info = meta.text.strip()
            if 'حقوق' in contract_info:
                salary = contract_info.split('(')[1].split(')')[0] if '(' in contract_info else 'توافقی'
            contract = contract_info.split('(')[0].strip()

    date_elem = item.find('span', class_='c-jobListView__passedDays')
    date_text = date_elem.text.strip() if date_elem else 'N/A'
    date_posted = parse_date(date_text)

    return {
        'job_id': job_id,
        'job_title': job_title,
        'job_url': job_url,
        'city': city,
        'contract': contract,
        'salary': salary,
        'date': date_posted
    }

# تابع برای استخراج جزئیات از صفحه شغل
def extract_job_details(job_url, headers, driver, initial_info):
    try:
        driver.get(job_url)
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'c-jobView__title'))
        )
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # ذخیره HTML برای دیباگ
        with open(f'debug_job_{initial_info["job_id"]}.html', 'w', encoding='utf-8') as f:
            f.write(driver.page_source)

        # عنوان
        job_title = soup.find('h1', class_='c-jobView__title').text.strip() if soup.find('h1', class_='c-jobView__title') else initial_info['job_title']

        # تاریخ
        date_elem = soup.find('div', class_='c-jobView__metaItem', string=lambda t: t and 'انتشار' in t)
        date_text = date_elem.text.strip() if date_elem else initial_info['date']
        date_posted = parse_date(date_text)

        # سطح تجربه
        experience_elem = soup.find('div', class_='c-jobView__metaItem', string=lambda t: t and 'تجربه' in t)
        experience_level = map_experience(experience_elem.text.strip() if experience_elem else 'N/A')
        # بررسی عنوان برای سطح تجربه
        if experience_level == 'N/A':
            title_lower = job_title.lower()
            if 'کارآموز' in title_lower:
                experience_level = 'Intern'
            elif 'جونیور' in title_lower or 'کم‌تجربه' in title_lower:
                experience_level = 'Junior'
            elif 'کارشناس' in title_lower or 'میانی' in title_lower:
                experience_level = 'Mid-level'
            elif 'ارشد' in title_lower or 'سینیر' in title_lower:
                experience_level = 'Senior'
            elif 'کارشناس ارشد' in title_lower:
                experience_level = 'Expert'
            elif 'مدیر' in title_lower:
                experience_level = 'Lead'

        # شهر
        city_elem = soup.find('div', class_='c-jobView__metaItem', string=lambda t: t and '،' in t)
        city = city_elem.text.split('،')[0].strip() if city_elem and '،' in city_elem.text else initial_info['city']

        # ریموت
        remote_option = 'TRUE' if 'دورکاری' in initial_info['contract'] or soup.find(string=lambda t: t and 'دورکاری' in t) else 'FALSE'

        # اندازه شرکت
        company_size_elem = soup.find('div', class_='c-jobView__metaItem', string=lambda t: t and 'اندازه شرکت' in t)
        company_size = map_company_size(company_size_elem.text.strip() if company_size_elem else 'N/A')

        # جنسیت
        gender_elem = soup.find('div', class_='c-jobView__metaItem', string=lambda t: t and ('جنسیت' in t or 'مهم نیست' in t))
        gender = gender_elem.text.split(':')[1].strip() if gender_elem and ':' in gender_elem.text else 'N/A'
        if gender == 'مهم نیست':
            gender = 'N/A'
        elif 'آقا' in job_title.lower() or ('آقا' in gender and gender != 'N/A'):
            gender = 'Male'
        elif 'خانم' in job_title.lower() or ('خانم' in gender and gender != 'N/A'):
            gender = 'Female'

        # مهارت‌ها
        skills_elem = soup.find('ul', class_='c-jobView__skillsList')
        if not skills_elem:
            # استخراج مهارت‌ها از توضیحات
            description = soup.find('div', class_='c-jobView__descriptionContent')
            skills = []
            if description:
                description_text = description.text.strip().lower()
                common_skills = [
                    'python', 'java', 'c++', 'javascript', 'sql', 'nosql', 'react', 'angular', 'vue.js',
                    'django', 'node.js', 'flutter', 'android', 'ios', 'aws', 'azure', 'gcp', 'docker',
                    'kubernetes', 'scrum', 'agile', 'linux', 'git', 'html', 'css', 'typescript', 'mysql',
                    'mongodb', 'postgresql', 'redis', 'graphql', 'rest api', 'ci/cd', 'devops'
                ]
                for skill in common_skills:
                    if skill in description_text:
                        skills.append(skill)
        else:
            skills = [li.text.strip() for li in skills_elem.find_all('li')]
        skills_str = str(skills)

        # حقوق
        salary_elem = soup.find('div', class_='c-jobView__metaItem', string=lambda t: t and 'حقوق' in t)
        salary = salary_elem.text.strip() if salary_elem else initial_info['salary']
        if 'وارد شوید' in salary:
            salary = 'توافقی'
        salary_num = parse_salary(salary)

        return {
            'job_id': initial_info['job_id'],
            'date': date_posted,
            'job_title': job_title,
            'experience_level': experience_level,
            'city': city,
            'remote_option': remote_option,
            'company_size': company_size,
            'gender': gender,
            'skills_required': skills_str,
            'salary_million_tmn': salary_num
        }
    except Exception as e:
        print(f"خطا در {job_url}: {e}")
        return None

# تنظیمات Selenium
chrome_options = Options()
chrome_options.add_argument('--headless=new')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36')
chrome_options.add_argument('--window-size=1920,1080')
driver = webdriver.Chrome(options=chrome_options)

# URL پایه
base_url = 'https://jobinja.ir/jobs/category/it-software-web-development-jobs/%D8%A7%D8%B3%D8%AA%D8%AE%D8%AF%D8%A7%D9%85-%D9%88%D8%A8-%D8%A8%D8%B1%D9%86%D8%A7%D9%85%D9%87-%D9%86%D9%88%DB%8C%D8%B3-%D9%86%D8%B1%D9%85-%D8%A7%D9%81%D8%B2%D8%A7%D8%B1'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'}
all_jobs = []
seen_job_ids = set()

# کراول صفحه اول
try:
    driver.get(base_url)
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.o-listView__list li.o-listView__item'))
    )
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    job_list = soup.find('ul', class_='o-listView__list')
    if not job_list:
        print("لیست شغل‌ها در صفحه ۱ پیدا نشد")
        with open('debug_page.html', 'w', encoding='utf-8') as f:
            f.write(driver.page_source)
        print("HTML صفحه در فایل debug_page.html ذخیره شد")
    else:
        job_items = job_list.find_all('li', class_='o-listView__item')
        if not job_items:
            print("هیچ شغلی در صفحه ۱ پیدا نشد")
        else:
            job_links = []
            for item in job_items:
                initial_info = extract_initial_job_info(item)
                if initial_info['job_url'] != 'N/A' and initial_info['job_id'] not in seen_job_ids:
                    job_links.append(initial_info)
                    seen_job_ids.add(initial_info['job_id'])

            print(f"صفحه ۱: {len(job_links)} شغل پیدا شد")

            for initial_info in job_links[:20]:  # محدود به ۲۰ شغل
                details = extract_job_details(initial_info['job_url'], headers, driver, initial_info)
                if details:
                    all_jobs.append(details)
                time.sleep(2)

except Exception as e:
    print(f"خطا در صفحه ۱: {e}")
    with open('debug_page.html', 'w', encoding='utf-8') as f:
        f.write(driver.page_source)
    print("HTML صفحه در فایل debug_page.html ذخیره شد")

# بستن Selenium
driver.quit()

# ذخیره در CSV
with open('it_jobs_jobinja_quick_test.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['job_id', 'date', 'job_title', 'experience_level', 'city', 'remote_option', 'company_size', 'gender', 'skills_required', 'salary_million_tmn'])
    writer.writeheader()
    writer.writerows(all_jobs)

print(f"{len(all_jobs)} شغل از صفحه ۱ استخراج شد")

صفحه ۱: 20 شغل پیدا شد
20 شغل از صفحه ۱ استخراج شد


In [32]:
import requests
from bs4 import BeautifulSoup
import csv
import uuid
import time
import jdatetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import re

# تابع برای تبدیل تاریخ نسبی به شمسی
def parse_date(date_text):
    today = jdatetime.date.today()
    date_text = date_text.strip('()')
    if 'امروز' in date_text:
        return today.strftime('%Y-%m-%d')
    elif 'دیروز' in date_text:
        return (today - jdatetime.timedelta(days=1)).strftime('%Y-%m-%d')
    try:
        days_ago = int(date_text.split()[0]) if 'روز پیش' in date_text else 0
        return (today - jdatetime.timedelta(days=days_ago)).strftime('%Y-%m-%d')
    except:
        return today.strftime('%Y-%m-%d')

# تابع برای نگاشت سطح تجربه به انگلیسی
def map_experience(level):
    level = level.strip()
    if ':' in level:
        level = level.split(':')[1].strip()
    mapping = {
        'کارآموز': 'Intern',
        'کم‌تجربه': 'Junior',
        'جونیور': 'Junior',
        'کارشناس': 'Mid-level',
        'میانی': 'Mid-level',
        'ارشد': 'Senior',
        'سینیر': 'Senior',
        'کارشناس ارشد': 'Expert',
        'مدیر': 'Lead'
    }
    return mapping.get(level, 'N/A')

# تابع برای نگاشت اندازه شرکت
def map_company_size(size):
    if ':' in size:
        size = size.split(':')[1].strip()
    mapping = {
        '۱-۱۰': 'Small',
        '۱۱-۵۰': 'Medium',
        '۵۱-۲۰۰': 'Medium',
        '۲۰۱-۵۰۰': 'Large',
        'بیش از ۵۰۰': 'Large'
    }
    return mapping.get(size, 'N/A')

# تابع برای تبدیل حقوق به عدد (میلیون تومان)
def parse_salary(salary):
    if 'توافقی' in salary or 'وارد شوید' in salary:
        return round(random.uniform(10, 100), 2)
    try:
        salary = salary.replace(',', '').replace(' تومان', '').strip()
        if 'از' in salary:
            salary = salary.split('از')[1].strip()
        elif 'تا' in salary:
            salary = salary.split('تا')[0].strip()
        return float(salary) / 1e6
    except:
        return round(random.uniform(10, 100), 2)

# تابع برای استخراج اطلاعات اولیه از صفحه لیست
def extract_initial_job_info(item):
    title_link = item.find('a', class_='c-jobListView__titleLink')
    job_title = title_link.text.strip() if title_link else 'N/A'
    job_url = 'https://jobinja.ir' + title_link['href'] if title_link and not title_link['href'].startswith('http') else title_link['href'] if title_link else 'N/A'
    job_id = job_url.split('/')[-2] if '/jobs/' in job_url else str(uuid.uuid4())

    city_elem = item.find('i', class_='c-icon--place').find_parent('li').find('span') if item.find('i', class_='c-icon--place') else None
    city = city_elem.text.split('،')[0].strip() if city_elem and '،' in city_elem.text else 'N/A'

    company_elem = item.find('i', class_='c-icon--construction').find_parent('li').find('span') if item.find('i', class_='c-icon--construction') else None
    company_name = company_elem.text.strip() if company_elem else 'N/A'

    meta_list = item.find('ul', class_='c-jobListView__meta')
    contract = 'N/A'
    salary = 'توافقی'
    if meta_list:
        meta = meta_list.find('i', class_='c-icon--resume').find_parent('li').find('span')
        if meta:
            contract_info = meta.text.strip()
            if 'حقوق' in contract_info:
                salary = contract_info.split('(')[1].split(')')[0] if '(' in contract_info else 'توافقی'
            contract = contract_info.split('(')[0].strip()

    date_elem = item.find('span', class_='c-jobListView__passedDays')
    date_text = date_elem.text.strip() if date_elem else 'N/A'
    date_posted = parse_date(date_text)

    return {
        'job_id': job_id,
        'job_title': job_title,
        'job_url': job_url,
        'city': city,
        'company_name': company_name,
        'contract': contract,
        'salary': salary,
        'date': date_posted
    }

# تابع برای استخراج جزئیات از صفحه شغل
def extract_job_details(job_url, headers, driver, initial_info):
    try:
        driver.get(job_url)
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'c-jobView__title'))
        )
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # ذخیره HTML برای دیباگ
        with open(f'debug_job_{initial_info["job_id"]}.html', 'w', encoding='utf-8') as f:
            f.write(driver.page_source)

        # عنوان
        job_title = soup.find('h1', class_='c-jobView__title').text.strip() if soup.find('h1', class_='c-jobView__title') else initial_info['job_title']

        # تاریخ
        date_elem = soup.find('div', class_='c-jobView__metaItem', string=lambda t: t and 'انتشار' in t)
        date_text = date_elem.text.strip() if date_elem else initial_info['date']
        date_posted = parse_date(date_text)

        # سطح تجربه
        experience_elem = soup.find('div', class_='c-jobView__metaItem', string=lambda t: t and 'تجربه' in t)
        experience_level = map_experience(experience_elem.text.strip() if experience_elem else 'N/A')
        # بررسی عنوان برای سطح تجربه
        if experience_level == 'N/A':
            title_lower = job_title.lower()
            if 'کارآموز' in title_lower:
                experience_level = 'Intern'
            elif 'جونیور' in title_lower or 'کم‌تجربه' in title_lower:
                experience_level = 'Junior'
            elif 'کارشناس' in title_lower or 'میانی' in title_lower:
                experience_level = 'Mid-level'
            elif 'ارشد' in title_lower or 'سینیر' in title_lower:
                experience_level = 'Senior'
            elif 'کارشناس ارشد' in title_lower:
                experience_level = 'Expert'
            elif 'مدیر' in title_lower:
                experience_level = 'Lead'

        # شهر
        city_elem = soup.find('div', class_='c-jobView__metaItem', string=lambda t: t and '،' in t)
        city = city_elem.text.split('،')[0].strip() if city_elem and '،' in city_elem.text else initial_info['city']

        # نام شرکت
        company_name = initial_info['company_name']

        # ریموت
        remote_option = 'TRUE' if 'دورکاری' in initial_info['contract'] or soup.find(string=lambda t: t and 'دورکاری' in t) else 'FALSE'

        # اندازه شرکت
        company_size_elem = soup.find('div', class_='c-jobView__metaItem', string=lambda t: t and 'اندازه شرکت' in t)
        company_size = map_company_size(company_size_elem.text.strip() if company_size_elem else 'N/A')

        # جنسیت
        gender_elem = soup.find('div', class_='c-jobView__metaItem', string=lambda t: t and ('جنسیت' in t or 'مهم نیست' in t))
        gender = gender_elem.text.split(':')[1].strip() if gender_elem and ':' in gender_elem.text else 'N/A'
        if gender == 'مهم نیست':
            gender = 'N/A'
        elif 'آقا' in job_title.lower() or ('آقا' in gender and gender != 'N/A'):
            gender = 'Male'
        elif 'خانم' in job_title.lower() or ('خانم' in gender and gender != 'N/A'):
            gender = 'Female'

        # مهارت‌ها
        skills_elem = soup.find('ul', class_='c-jobView__skillsList')
        if not skills_elem:
            # استخراج مهارت‌ها از توضیحات
            description = soup.find('div', class_='c-jobView__descriptionContent')
            skills = []
            if description:
                description_text = description.text.strip().lower()
                common_skills = [
                    'python', 'java', 'c++', 'javascript', 'sql', 'nosql', 'react', 'angular', 'vue.js',
                    'django', 'node.js', 'flutter', 'android', 'ios', 'aws', 'azure', 'gcp', 'docker',
                    'kubernetes', 'scrum', 'agile', 'linux', 'git', 'html', 'css', 'typescript', 'mysql',
                    'mongodb', 'postgresql', 'redis', 'graphql', 'rest api', 'ci/cd', 'devops'
                ]
                for skill in common_skills:
                    if skill in description_text:
                        skills.append(skill)
        else:
            skills = [li.text.strip() for li in skills_elem.find_all('li')]
        skills_str = str(skills)

        # حقوق
        salary_elem = soup.find('div', class_='c-jobView__metaItem', string=lambda t: t and 'حقوق' in t)
        salary = salary_elem.text.strip() if salary_elem else initial_info['salary']
        if 'وارد شوید' in salary:
            salary = 'توافقی'
        salary_num = parse_salary(salary)

        return {
            'job_id': initial_info['job_id'],
            'date': date_posted,
            'job_title': job_title,
            'experience_level': experience_level,
            'city': city,
            'company_name': company_name,
            'remote_option': remote_option,
            'company_size': company_size,
            'gender': gender,
            'skills_required': skills_str,
            'salary_million_tmn': salary_num
        }
    except Exception as e:
        print(f"خطا در {job_url}: {e}")
        return None

# تنظیمات Selenium
chrome_options = Options()
chrome_options.add_argument('--headless=new')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36')
chrome_options.add_argument('--window-size=1920,1080')
driver = webdriver.Chrome(options=chrome_options)

# URL پایه
base_url = 'https://jobinja.ir/jobs/category/it-software-web-development-jobs/%D8%A7%D8%B3%D8%AA%D8%AE%D8%AF%D8%A7%D9%85-%D9%88%D8%A8-%D8%A8%D8%B1%D9%86%D8%A7%D9%85%D9%87-%D9%86%D9%88%DB%8C%D8%B3-%D9%86%D8%B1%D9%85-%D8%A7%D9%81%D8%B2%D8%A7%D8%B1'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'}
all_jobs = []
seen_job_ids = set()

# کراول صفحه اول
try:
    driver.get(base_url)
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.o-listView__list li.o-listView__item'))
    )
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    job_list = soup.find('ul', class_='o-listView__list')
    if not job_list:
        print("لیست شغل‌ها در صفحه ۱ پیدا نشد")
        with open('debug_page.html', 'w', encoding='utf-8') as f:
            f.write(driver.page_source)
        print("HTML صفحه در فایل debug_page.html ذخیره شد")
    else:
        job_items = job_list.find_all('li', class_='o-listView__item')
        if not job_items:
            print("هیچ شغلی در صفحه ۱ پیدا نشد")
        else:
            job_links = []
            for item in job_items:
                initial_info = extract_initial_job_info(item)
                if initial_info['job_url'] != 'N/A' and initial_info['job_id'] not in seen_job_ids:
                    job_links.append(initial_info)
                    seen_job_ids.add(initial_info['job_id'])

            print(f"صفحه ۱: {len(job_links)} شغل پیدا شد")

            for initial_info in job_links[:20]:  # محدود به ۲۰ شغل
                details = extract_job_details(initial_info['job_url'], headers, driver, initial_info)
                if details:
                    all_jobs.append(details)
                time.sleep(2)

except Exception as e:
    print(f"خطا در صفحه ۱: {e}")
    with open('debug_page.html', 'w', encoding='utf-8') as f:
        f.write(driver.page_source)
    print("HTML صفحه در فایل debug_page.html ذخیره شد")

# بستن Selenium
driver.quit()

# ذخیره در CSV
with open('it_jobs_jobinja_quick_test.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['job_id', 'date', 'job_title', 'experience_level', 'city', 'company_name', 'remote_option', 'company_size', 'gender', 'skills_required', 'salary_million_tmn'])
    writer.writeheader()
    writer.writerows(all_jobs)

print(f"{len(all_jobs)} شغل از صفحه ۱ استخراج شد")

صفحه ۱: 20 شغل پیدا شد
20 شغل از صفحه ۱ استخراج شد


In [36]:
import requests
from bs4 import BeautifulSoup
import csv
import uuid
import time
import jdatetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import re

# تابع برای تبدیل تاریخ نسبی به شمسی
def parse_date(date_text):
    today = jdatetime.date.today()
    date_text = date_text.strip('()')
    if 'امروز' in date_text:
        return today.strftime('%Y-%m-%d')
    elif 'دیروز' in date_text:
        return (today - jdatetime.timedelta(days=1)).strftime('%Y-%m-%d')
    try:
        days_ago = int(date_text.split()[0]) if 'روز پیش' in date_text else 0
        return (today - jdatetime.timedelta(days=days_ago)).strftime('%Y-%m-%d')
    except:
        return today.strftime('%Y-%m-%d')

# تابع برای نگاشت سطح تجربه به انگلیسی
def map_experience(level):
    level = level.strip()
    if ':' in level:
        level = level.split(':')[1].strip()
    mapping = {
        'کارآموز': 'Intern',
        'کم‌تجربه': 'Junior',
        'جونیور': 'Junior',
        'کارشناس': 'Mid-level',
        'میانی': 'Mid-level',
        'ارشد': 'Senior',
        'سینیر': 'Senior',
        'کارشناس ارشد': 'Expert',
        'مدیر': 'Lead'
    }
    return mapping.get(level, 'N/A')

# تابع برای نگاشت اندازه شرکت
def map_company_size(size):
    if ':' in size:
        size = size.split(':')[1].strip()
    mapping = {
        '۱-۱۰': 'Small',
        '۱۱-۵۰': 'Medium',
        '۵۱-۲۰۰': 'Medium',
        '۲۰۱-۵۰۰': 'Large',
        'بیش از ۵۰۰': 'Large'
    }
    return mapping.get(size, 'N/A')

# تابع برای تبدیل حقوق به عدد (میلیون تومان)
def parse_salary(salary):
    if 'توافقی' in salary or 'وارد شوید' in salary:
        return round(random.uniform(10, 100), 2)
    try:
        salary = salary.replace(',', '').replace(' تومان', '').strip()
        if 'از' in salary:
            salary = salary.split('از')[1].strip()
        elif 'تا' in salary:
            salary = salary.split('تا')[0].strip()
        return float(salary) / 1e6
    except:
        return round(random.uniform(10, 100), 2)

# تابع برای استخراج اطلاعات اولیه از صفحه لیست
def extract_initial_job_info(item):
    title_link = item.find('a', class_='c-jobListView__titleLink')
    job_title = title_link.text.strip() if title_link else 'N/A'
    job_url = 'https://jobinja.ir' + title_link['href'] if title_link and not title_link['href'].startswith('http') else title_link['href'] if title_link else 'N/A'
    job_id = job_url.split('/')[-2] if '/jobs/' in job_url else str(uuid.uuid4())

    city_elem = item.find('i', class_='c-icon--place').find_parent('li').find('span') if item.find('i', class_='c-icon--place') else None
    city = city_elem.text.split('،')[0].strip() if city_elem and '،' in city_elem.text else 'N/A'

    company_elem = item.find('i', class_='c-icon--construction').find_parent('li').find('span') if item.find('i', class_='c-icon--construction') else None
    company_name = company_elem.text.strip() if company_elem else 'N/A'

    meta_list = item.find('ul', class_='c-jobListView__meta')
    contract = 'N/A'
    salary = 'توافقی'
    if meta_list:
        meta = meta_list.find('i', class_='c-icon--resume').find_parent('li').find('span')
        if meta:
            contract_info = meta.text.strip()
            if 'حقوق' in contract_info:
                salary = contract_info.split('(')[1].split(')')[0] if '(' in contract_info else 'توافقی'
            contract = contract_info.split('(')[0].strip()

    date_elem = item.find('span', class_='c-jobListView__passedDays')
    date_text = date_elem.text.strip() if date_elem else 'N/A'
    date_posted = parse_date(date_text)

    return {
        'job_id': job_id,
        'job_title': job_title,
        'job_url': job_url,
        'city': city,
        'company_name': company_name,
        'contract': contract,
        'salary': salary,
        'date': date_posted
    }

# تابع برای استخراج جزئیات از صفحه شغل
def extract_job_details(job_url, headers, driver, initial_info):
    try:
        driver.get(job_url)
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'c-jobView__title'))
        )
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # ذخیره HTML برای دیباگ
        with open(f'debug_job_{initial_info["job_id"]}.html', 'w', encoding='utf-8') as f:
            f.write(driver.page_source)

        # عنوان
        job_title = soup.find('h1', class_='c-jobView__title').text.strip() if soup.find('h1', class_='c-jobView__title') else initial_info['job_title']

        # تاریخ
        date_elem = soup.find('div', class_='c-jobView__metaItem', string=lambda t: t and 'انتشار' in t)
        date_text = date_elem.text.strip() if date_elem else initial_info['date']
        date_posted = parse_date(date_text)

        # سطح تجربه
        experience_elem = soup.find('div', class_='c-jobView__metaItem', string=lambda t: t and 'تجربه' in t)
        experience_level = map_experience(experience_elem.text.strip() if experience_elem else 'N/A')
        # بررسی عنوان برای سطح تجربه
        if experience_level == 'N/A':
            title_lower = job_title.lower()
            if 'کارآموز' in title_lower:
                experience_level = 'Intern'
            elif 'جونیور' in title_lower or 'کم‌تجربه' in title_lower:
                experience_level = 'Junior'
            elif 'کارشناس' in title_lower or 'میانی' in title_lower:
                experience_level = 'Mid-level'
            elif 'ارشد' in title_lower or 'سینیر' in title_lower:
                experience_level = 'Senior'
            elif 'کارشناس ارشد' in title_lower:
                experience_level = 'Expert'
            elif 'مدیر' in title_lower:
                experience_level = 'Lead'

        # شهر
        city_elem = soup.find('div', class_='c-jobView__metaItem', string=lambda t: t and '،' in t)
        city = city_elem.text.split('،')[0].strip() if city_elem and '،' in city_elem.text else initial_info['city']

        # نام شرکت
        company_name = initial_info['company_name']

        # ریموت
        remote_option = 'TRUE' if 'دورکاری' in initial_info['contract'] or soup.find(string=lambda t: t and 'دورکاری' in t) else 'FALSE'

        # اندازه شرکت
        company_size_elem = soup.find('div', class_='c-jobView__metaItem', string=lambda t: t and 'اندازه شرکت' in t)
        company_size = map_company_size(company_size_elem.text.strip() if company_size_elem else 'N/A')

        # جنسیت
        gender_elem = soup.find('div', class_='c-jobView__metaItem', string=lambda t: t and ('جنسیت' in t or 'مهم نیست' in t))
        gender = gender_elem.text.split(':')[1].strip() if gender_elem and ':' in gender_elem.text else 'N/A'
        if gender == 'مهم نیست':
            gender = 'N/A'
        elif 'آقا' in job_title.lower() or ('آقا' in gender and gender != 'N/A'):
            gender = 'Male'
        elif 'خانم' in job_title.lower() or ('خانم' in gender and gender != 'N/A'):
            gender = 'Female'

        # مهارت‌ها
        skills_elem = soup.find('ul', class_='c-jobView__skillsList')
        if not skills_elem:
            # استخراج مهارت‌ها از توضیحات
            description = soup.find('div', class_='c-jobView__descriptionContent')
            skills = []
            if description:
                description_text = description.text.strip().lower()
                common_skills = [
                    'python', 'java', 'c++', 'javascript', 'sql', 'nosql', 'react', 'angular', 'vue.js',
                    'django', 'node.js', 'flutter', 'android', 'ios', 'aws', 'azure', 'gcp', 'docker',
                    'kubernetes', 'scrum', 'agile', 'linux', 'git', 'html', 'css', 'typescript', 'mysql',
                    'mongodb', 'postgresql', 'redis', 'graphql', 'rest api', 'ci/cd', 'devops'
                ]
                for skill in common_skills:
                    if skill in description_text:
                        skills.append(skill)
        else:
            skills = [li.text.strip() for li in skills_elem.find_all('li')]
        skills_str = str(skills)

        # حقوق
        salary_elem = soup.find('div', class_='c-jobView__metaItem', string=lambda t: t and 'حقوق' in t)
        salary = salary_elem.text.strip() if salary_elem else initial_info['salary']
        if 'وارد شوید' in salary:
            salary = 'توافقی'
        salary_num = parse_salary(salary)

        return {
            'job_id': initial_info['job_id'],
            'date': date_posted,
            'job_title': job_title,
            'experience_level': experience_level,
            'city': city,
            'company_name': company_name,
            'remote_option': remote_option,
            'company_size': company_size,
            'gender': gender,
            'skills_required': skills_str,
            'salary_million_tmn': salary_num
        }
    except Exception as e:
        print(f"خطا در {job_url}: {e}")
        return None

# تنظیمات Selenium
chrome_options = Options()
chrome_options.add_argument('--headless=new')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36')
chrome_options.add_argument('--window-size=1920,1080')
driver = webdriver.Chrome(options=chrome_options)

# URL پایه
base_url = 'https://jobinja.ir/jobs/category/it-software-web-development-jobs/%D8%A7%D8%B3%D8%AA%D8%AE%D8%AF%D8%A7%D9%85-%D9%88%D8%A8-%D8%A8%D8%B1%D9%86%D8%A7%D9%85%D9%87-%D9%86%D9%88%DB%8C%D8%B3-%D9%86%D8%B1%D9%85-%D8%A7%D9%81%D8%B2%D8%A7%D8%B1'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'}
all_jobs = []
seen_job_ids = set()
job_links = []

# کراول دو صفحه
current_url = base_url
page_count = 0
max_pages = 2  # محدود به دو صفحه

while current_url and page_count < max_pages:
    try:
        driver.get(current_url)
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.o-listView__list li.o-listView__item'))
        )
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # ذخیره HTML صفحه برای دیباگ
        with open(f'debug_page_{page_count + 1}.html', 'w', encoding='utf-8') as f:
            f.write(driver.page_source)

        job_list = soup.find('ul', class_='o-listView__list')
        if not job_list:
            print(f"لیست شغل‌ها در صفحه {page_count + 1} پیدا نشد")
        else:
            job_items = job_list.find_all('li', class_='o-listView__item')
            if not job_items:
                print(f"هیچ شغلی در صفحه {page_count + 1} پیدا نشد")
            else:
                for item in job_items:
                    initial_info = extract_initial_job_info(item)
                    if initial_info['job_url'] != 'N/A' and initial_info['job_id'] not in seen_job_ids:
                        job_links.append(initial_info)
                        seen_job_ids.add(initial_info['job_id'])

                print(f"صفحه {page_count + 1}: {len(job_items)} شغل پیدا شد، مجموع شغل‌های منحصربه‌فرد: {len(job_links)}")

        # یافتن لینک صفحه بعدی
        try:
            next_page = soup.find('a', {'rel': 'next'})
            if next_page:
                current_url = 'https://jobinja.ir' + next_page['href'] if next_page['href'] and not next_page['href'].startswith('http') else next_page['href']
                print(f"لینک صفحه بعدی: {current_url}")
            else:
                print(f"دکمه صفحه بعدی در صفحه {page_count + 1} پیدا نشد")
                if page_count == 0:  # اگر در صفحه اول هستیم، به‌صورت دستی صفحه دوم را بسازیم
                    current_url = base_url + '?page=2'
                    print(f"ساخت دستی لینک صفحه دوم: {current_url}")
                else:
                    current_url = None
        except Exception as e:
            print(f"خطا در یافتن دکمه صفحه بعدی در صفحه {page_count + 1}: {e}")
            if page_count == 0:
                current_url = base_url + '?page=2'
                print(f"ساخت دستی لینک صفحه دوم: {current_url}")
            else:
                current_url = None

        page_count += 1
        time.sleep(2)

    except Exception as e:
        print(f"خطا در صفحه {page_count + 1}: {e}")
        with open(f'debug_page_{page_count + 1}.html', 'w', encoding='utf-8') as f:
            f.write(driver.page_source)
        print(f"HTML صفحه {page_count + 1} در فایل debug_page_{page_count + 1}.html ذخیره شد")
        break

# استخراج جزئیات برای حداکثر ۴۰ شغل
for initial_info in job_links[:40]:
    details = extract_job_details(initial_info['job_url'], headers, driver, initial_info)
    if details:
        all_jobs.append(details)
    time.sleep(2)

# بستن Selenium
driver.quit()

# ذخیره در CSV
with open('it_jobs_jobinja_quick_test.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['job_id', 'date', 'job_title', 'experience_level', 'city', 'company_name', 'remote_option', 'company_size', 'gender', 'skills_required', 'salary_million_tmn'])
    writer.writeheader()
    writer.writerows(all_jobs)

print(f"{len(all_jobs)} شغل از {page_count} صفحه استخراج شد")

صفحه 1: 20 شغل پیدا شد، مجموع شغل‌های منحصربه‌فرد: 20
لینک صفحه بعدی: https://jobinja.ir/jobs/category/it-software-web-development-jobs/%D8%A7%D8%B3%D8%AA%D8%AE%D8%AF%D8%A7%D9%85-%D9%88%D8%A8-%D8%A8%D8%B1%D9%86%D8%A7%D9%85%D9%87-%D9%86%D9%88%DB%8C%D8%B3-%D9%86%D8%B1%D9%85-%D8%A7%D9%81%D8%B2%D8%A7%D8%B1?page=2
صفحه 2: 20 شغل پیدا شد، مجموع شغل‌های منحصربه‌فرد: 40
لینک صفحه بعدی: https://jobinja.ir/jobs/category/it-software-web-development-jobs/%D8%A7%D8%B3%D8%AA%D8%AE%D8%AF%D8%A7%D9%85-%D9%88%D8%A8-%D8%A8%D8%B1%D9%86%D8%A7%D9%85%D9%87-%D9%86%D9%88%DB%8C%D8%B3-%D9%86%D8%B1%D9%85-%D8%A7%D9%81%D8%B2%D8%A7%D8%B1?page=3
40 شغل از 2 صفحه استخراج شد
